In [1]:
import sys
from Bio import Entrez
# *Always* tell NCBI who you are
Entrez.email = "szong@bcgsc.ca"
import pandas as pd

In [2]:
def retrieve_annotation(id_list):

    """Annotates Entrez Gene IDs using Bio.Entrez, in particular epost (to
    submit the data to NCBI) and esummary to retrieve the information.
    Returns a list of dictionaries with the annotations."""

    request = Entrez.epost("gene",id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        #FIXME: How generate NAs instead of causing an error with invalid IDs?
        print("An error occurred while retrieving the annotations.")
        print("The error returned was %s" % e)
        sys.exit(-1)

    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db="gene", webenv=webEnv, query_key =
            queryKey)
    annotations = Entrez.read(data)

    print("Retrieved %d annotations for %d genes" % (len(annotations), len(id_list)))

    return annotations

In [80]:
genes = ['2066', '83595', '64919', '51741', '196528', '100288687', '53353', '2272', '339967', '657', '64784', '8626', '3094', '54984', '27436', '2064', '6794', '4609', '7015', '3728', '3164']

In [3]:
genes = ['339967']

In [4]:
# dict = retrieve_annotation(['2066', '83595', '64919', '51741', '196528', '100288687', '53353', '2272', '339967', '657', '64784', '8626', '3094', '54984', '27436', '2064', '6794', '4609', '7015', '3728', '3164'
# ])
for gene in genes:
    dict = retrieve_annotation([gene])
    txt = str(dict.get('DocumentSummarySet')).split('\'Summary\': ')[1].split('\'ChrSort\'')[0]
    print(gene, txt)


Retrieved 1 annotations for 1 genes
339967 '', 


In [65]:
type(dict)`


Bio.Entrez.Parser.DictionaryElement

In [79]:
str(dict.get('DocumentSummarySet')).split('\'Summary\': ')[1].split('\'ChrSort\'')[0]

"'This gene is a member of the Tyr protein kinase family and the epidermal growth factor receptor subfamily. It encodes a single-pass type I membrane protein with multiple cysteine rich domains, a transmembrane domain, a tyrosine kinase domain, a phosphotidylinositol-3 kinase binding site and a PDZ domain binding motif. The protein binds to and is activated by neuregulins and other factors and induces a variety of cellular responses including mitogenesis and differentiation. Multiple proteolytic events allow for the release of a cytoplasmic fragment and an extracellular fragment. Mutations in this gene have been associated with cancer. Alternatively spliced variants which encode different protein isoforms have been described; however, not all variants have been fully characterized. [provided by RefSeq, Jul 2008]', "

In [73]:

for i in Entrez.parse(dict, validate=True):
    print(i)

AttributeError: 'DictionaryElement' object has no attribute 'read'

In [54]:
f = '/home/szong/projects/resource/ens69/gene_info_ens69_nina.txt'
df = pd.read_csv(f, sep='\t', header=None, usecols=[2,4])
df.head()


,2,4
0,SCYL3,SCY1-like_3_(S._cerevisiae)_[Source:HGNC_Symbo...
1,C1orf112,chromosome_1_open_reading_frame_112_[Source:HG...
2,FGR,Gardner-Rasheed_feline_sarcoma_viral_(v-fgr)_o...
3,CFH,complement_factor_H_[Source:HGNC_Symbol;Acc:4883]
4,STPG1,sperm-tail_PG-rich_repeat_containing_1_[Source...


In [55]:
lst = ['ERBB4', 'TERT', 'SOX7', 'BCL11B', 'WWOX', 'ARID2', 'DUX4', 'LRP1B', 'FHIT', 'JUP', 'TMPRSS11A', 'BMPR1A', 'CRTC3', 'TP63', 'ERBB2', 'STK11', 'HINT1', 'PINX1', 'EML4', 'MYC', 'NR4A1']

In [63]:
df2 = df[df[2].isin(lst)].drop_duplicates()

In [58]:
len(lst)

21

In [64]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/oncogenes_annotation.txt'
df2.to_csv(of, index=False)

In [49]:
def print_data(annotation):
    for gene_data in annotation:
        print('xxx', gene_data, type(gene_data))
        gene_id = gene_data["Id"]
        gene_symbol = gene_data["NomenclatureSymbol"]
        gene_name = gene_data["Description"]
        print("ID: %s - Gene Symbol: %s - Gene Name: %s" % (gene_id, gene_symbol, gene_name))

In [50]:
print_data(dict)

xxx DocumentSummarySet <class 'str'>


TypeError: string indices must be integers

In [25]:
pd.DataFrame(dict)

,DocumentSummarySet
DbBuild,Build180221-0050m.1
DocumentSummary,"[{'Name': 'ERBB4', 'Description': 'erb-b2 rece..."


In [31]:
annotation = retrieve_annotation(['2066'])

Retrieved 1 annotations for 1 genes


In [35]:
annotation

{'DocumentSummarySet': DictElement({'DocumentSummary': [DictElement({'Name': 'ERBB4', 'Description': 'erb-b2 receptor tyrosine kinase 4', 'Status': '0', 'CurrentID': '0', 'Chromosome': '2', 'GeneticSource': 'genomic', 'MapLocation': '2q34', 'OtherAliases': 'ALS19, HER4, p180erbB4', 'OtherDesignations': 'receptor tyrosine-protein kinase erbB-4|ERBB4 transcript variant I12DEL|ERBB4 transcript variant I20DEL|avian erythroblastic leukemia viral (v-erb-b2) oncogene homolog 4|human epidermal growth factor receptor 4|proto-oncogene-like protein c-ErbB-4|tyrosine kinase-type cell surface receptor HER4|v-erb-a erythroblastic leukemia viral oncogene homolog 4|v-erb-b2 avian erythroblastic leukemia viral oncogene homolog 4', 'NomenclatureSymbol': 'ERBB4', 'NomenclatureName': 'erb-b2 receptor tyrosine kinase 4', 'NomenclatureStatus': 'Official', 'Mim': ['600543'], 'GenomicInfo': [{'ChrLoc': '2', 'ChrAccVer': 'NC_000002.12', 'ChrStart': '212539146', 'ChrStop': '211375716', 'ExonCount': '31'}], 'Gen

In [39]:
def parse_genes(genes):
    """Parse various gene information including:
    1. Species name (taxonomy name)
    2. Entrez gene ID
    3. Official symbol
    4. RefSeq IDs
    5. Offical full name
    Basically, just to go through the parsed xml data.... A big headache to figure it out...
    Return a list of dictionary.
    """
    gene_info_list = []
    for gene_data in genes:
        gene_info = {}
        # get entrez ID
        try:
            print('xxxxx', gene_info)
            gene_info["entrez_id"] = gene_data["Entrezgene_track-info"]["Gene-track"]["Gene-track_geneid"]
        except KeyError:
            gene_info["entrez_id"] = ""
            continue
        gene_info["refseq_ids"] = []
        for comment in gene_data.get("Entrezgene_comments",[]):
            # look for refSeq annotation
            if comment.get("Gene-commentary_heading",None) == "NCBI Reference Sequences (RefSeq)":
                # get sub-comments
                for subcomment in comment.get("Gene-commentary_comment",[]):
                    for product in subcomment.get("Gene-commentary_products",[]):
                        if product.get("Gene-commentary_heading",None) == "mRNA Sequence":
                            gene_info["refseq_ids"].append(product.get("Gene-commentary_accession",""))
        # get properties
        gene_info["official_symbol"] = "" # optional
        gene_info["official_full_name"] = "" # optional
        for gene_property in gene_data.get("Entrezgene_properties",[]):
            if gene_property.get("Gene-commentary_label",None) == "Nomenclature":
                for sub_property in gene_property["Gene-commentary_properties"]:
                    if sub_property.get("Gene-commentary_label",None)  == "Official Symbol":
                        gene_info["official_symbol"] = sub_property.get("Gene-commentary_text","")
                    if sub_property.get("Gene-commentary_label",None)  == "Official Full Name":
                        gene_info["official_full_name"] = sub_property.get("Gene-commentary_text","")

        # get taxname
        try:
            gene_info["taxname"] = gene_data["Entrezgene_source"]["BioSource"]["BioSource_org"]["Org-ref"]["Org-ref_taxname"]
        except KeyError:
            gene_info["taxname"] = ""
            continue
        gene_info_list.append(gene_info)

    return gene_info_list

In [40]:
parse_genes('2066')

xxxxx {}


TypeError: string indices must be integers

In [36]:
type(annotation['DocumentSummarySet'])

Bio.Entrez.Parser.StructureElement

In [10]:
#!/usr/bin/env python
# Time-stamp: <2011-05-05 18:31:45 Tao Liu>

"""Module Description
Copyright (c) 2011 Tao Liu <taoliu@jimmy.harvard.edu>
This code is free software; you can redistribute it and/or modify it
under the terms of the BSD License (see the file COPYING included with
the distribution).
@status:  experimental
@version: 0.1
@author:  Tao Liu
@contact: taoliu@jimmy.harvard.edu
"""

# ------------------------------------
# python modules
# ------------------------------------

import sys
from optparse import OptionParser
 
from Bio import Entrez
 
# ------------------------------------
# constants
# ------------------------------------
# *Always* tell NCBI who you are
Entrez.email = "your email here"

# ------------------------------------
# Misc functions
# ------------------------------------
def search_genes(id_list,search_field):
    """Use ESearch to convert RefSeq or Gene symbols to standard
    Entrez IDs.
    A request to esearch.cgi is like:
    http://www.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gene&term=ID_LIST[SEARCH_FIELD]
    Return a list of Entrez IDs.
    """
    term = " OR ".join(map(lambda x:x+"["+search_field+"]",id_list))
    esearch_result = Entrez.esearch(db="gene",term=term,retmod="xml")
    parsed_result = Entrez.read(esearch_result)
    return parsed_result['IdList']

def fetch_genes(id_list):
    """Fetch Entrez Gene records using Bio.Entrez, in particular epost
    (to submit the data to NCBI) and efetch to retrieve the
    information, then use Entrez.read to parse the data.
    Returns a list of parsed gene records.
    """
 
    request = Entrez.epost("gene",id=",".join(id_list))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        #FIXME: How generate NAs instead of causing an error with invalid IDs?
        print("An error occurred while retrieving the annotations.")
        print("The error returned was %s" % e)
        sys.exit(-1)
 
    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    efetch_result = Entrez.efetch(db="gene", webenv=webEnv, query_key = queryKey, retmode="xml")
    genes = Entrez.read(efetch_result)
    #print("Retrieved %d records for %d genes" % (len(genes),len(id_list))
    return genes

def parse_genes(genes):
    """Parse various gene information including:
    1. Species name (taxonomy name)
    2. Entrez gene ID
    3. Official symbol
    4. RefSeq IDs
    5. Offical full name
    Basically, just to go through the parsed xml data.... A big headache to figure it out...
    Return a list of dictionary.
    """
    gene_info_list = []
    for gene_data in genes:
        gene_info = {}
        # get entrez ID
        try:
            gene_info["entrez_id"] = gene_data["Entrezgene_track-info"]["Gene-track"]["Gene-track_geneid"]
        except KeyError:
            gene_info["entrez_id"] = ""
            continue
        gene_info["refseq_ids"] = []
        for comment in gene_data.get("Entrezgene_comments",[]):
            # look for refSeq annotation
            if comment.get("Gene-commentary_heading",None) == "NCBI Reference Sequences (RefSeq)":
                # get sub-comments
                for subcomment in comment.get("Gene-commentary_comment",[]):
                    for product in subcomment.get("Gene-commentary_products",[]):
                        if product.get("Gene-commentary_heading",None) == "mRNA Sequence":
                            gene_info["refseq_ids"].append(product.get("Gene-commentary_accession",""))
        # get properties
        gene_info["official_symbol"] = "" # optional
        gene_info["official_full_name"] = "" # optional
        for gene_property in gene_data.get("Entrezgene_properties",[]):
            if gene_property.get("Gene-commentary_label",None) == "Nomenclature":
                for sub_property in gene_property["Gene-commentary_properties"]:
                    if sub_property.get("Gene-commentary_label",None)  == "Official Symbol":
                        gene_info["official_symbol"] = sub_property.get("Gene-commentary_text","")
                    if sub_property.get("Gene-commentary_label",None)  == "Official Full Name":
                        gene_info["official_full_name"] = sub_property.get("Gene-commentary_text","")

        # get taxname
        try:
            gene_info["taxname"] = gene_data["Entrezgene_source"]["BioSource"]["BioSource_org"]["Org-ref"]["Org-ref_taxname"]
        except KeyError:
            gene_info["taxname"] = ""
            continue
        gene_info_list.append(gene_info)

    return gene_info_list

def print_genes (gene_info_list):
    """print(out parsed entrez gene information in tab-delimited way.
    """
    # header
    print("%s\t%s\t%s\t%s\t%s" % ("TaxonomyName","EntrezID","OfficialSymbol","RefSeqIDs","OfficialFullName"))
    for g in gene_info_list:
        print("%s\t%s\t%s\t%s\t%s" % (g["taxname"],g["entrez_id"],g["official_symbol"],",".join(g["refseq_ids"]),g["official_full_name"]))

# ------------------------------------
# Classes
# ------------------------------------

# ------------------------------------
# Main function
# ------------------------------------
def main():
    usage = "usage: %prog [options]"
    description = "Use NCBI web API to convert gene ids between different identifier types."
    
    optparser = OptionParser(version="%prog 0.1",description=description,usage=usage,add_help_option=False)
    optparser.add_option("-h","--help",action="help",help="Show this help message and exit.")
    optparser.add_option("-i","--id",dest="ids",type="string",action="append",
                         help="Gene id, according to identifier setting of input, can be Entrez, RefSeq, or Gene symbol. Multiple ids are allowed.")
    optparser.add_option("-a","--itype",dest="itype",default="entrez",
                         help="Identifier type of your input ids. Can be 'entrez', 'refseq', or 'symbol'. Default: 'entrez'.")
    (options,args) = optparser.parse_args()
    if not options.ids:
        optparser.print_help()
        sys.exit(-1)
    input_id_list = options.ids
    if options.itype == "refseq":
        entrez_id_list = search_genes(input_id_list,"ACCN")
    elif options.itype == "symbol":
        entrez_id_list = search_genes(input_id_list,"GENE")
    elif options.itype == "entrez":
        entrez_id_list = input_id_list
    
    entrez_id_genes = fetch_genes(entrez_id_list)
    parsed_genes = parse_genes(entrez_id_genes)
    print_genes(parsed_genes)

if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt:
        sys.stderr.write("User interrupt me! ;-) See you!\n")
        sys.exit(0)

Usage: ipykernel_launcher.py [options]

ipykernel_launcher.py: error: no such option: -f


SystemExit: 2

/home/szong/anaconda3/envs/szvenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [26]:
var =888

In [36]:
def test():
    global var
    var += 1
    print(var)

test()

894


In [17]:
globvar = 0

def set_globvar_to_one():
    global globvar    # Needed to modify global copy of globvar
    globvar = 1

def print_globvar():
    print(globvar)     # No need for global declaration to read value of globvar

set_globvar_to_one()
print_globvar()       # Prints 1

1


UnboundLocalError: local variable 'var' referenced before assignment